This is to support part V.A in our paper. We use vadersentiment to analyze stock sentiment. The result is to allocate each tweet as one of three types: positive, neutral, negative

In [ ]:
#!pip install vaderSentiment

In [ ]:
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# special matplotlib argument for improved plots
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 6

In [ ]:
company = pd.read_csv('/Users/../Company.csv')
tweet_id = pd.read_csv('/Users/../Company_Tweet.csv')
tweet = pd.read_csv('/Users/../Tweet.csv')
company_value = pd.read_csv('/Users/../CompanyValues.csv')

In [ ]:
company.head()

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc
3,AMZN,Amazon.com
4,TSLA,Tesla Inc


In [ ]:
tweet_id.head()

In [ ]:
tweet.head()

In [ ]:
company_value.head()

In [ ]:
tweet['post_date'] = pd.to_datetime(tweet['post_date'], unit='s')
tweet['post_date'] = pd.to_datetime(tweet['post_date'].apply(lambda date: date.date()))

In [ ]:
tweet.head()

In [ ]:
tweet_info = pd.merge(tweet_id, tweet, on='tweet_id', how='inner')
tweet_info.head()

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num
0,550803612197457920,AAPL,SentiQuant,2015-01-01,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
1,550803612197457920,AMZN,SentiQuant,2015-01-01,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1
2,550803610825928706,AAPL,SentiQuant,2015-01-01,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
3,550803610825928706,GOOGL,SentiQuant,2015-01-01,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1
4,550803610825928706,AMZN,SentiQuant,2015-01-01,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1


In [ ]:
tweet_info["total_engangement"] = tweet_info["comment_num"] + tweet_info["retweet_num"] + tweet_info["like_num"]
#Engagement will be further added to calculations, comments and retweets earn higher engagement score, can be changed

In [ ]:
#Create df based on company
Apple = tweet_info[(tweet_info.ticker_symbol == 'AAPL')]
#Apply sentiment analysis
Apple['sentiment_scores'] = Apple['body'].apply(lambda body: SentimentIntensityAnalyzer().polarity_scores(body))
#Get compound score
Apple['score'] = Apple['sentiment_scores'].apply(lambda score: "Positive" if score['compound'] >= 0.05 else 
                                                 ("Negative" if score['compound'] <= -0.05 else "Neutral"))
Apple.head()

In [ ]:
Amazon = tweet_info[(tweet_info.ticker_symbol == 'AMZN')]
Amazon['sentiment_scores'] = Amazon['body'].apply(lambda body: SentimentIntensityAnalyzer().polarity_scores(body))
Amazon['score'] = Amazon['sentiment_scores'].apply(lambda score: "Positive" if score['compound'] >= 0.05 else ("Negative" if score['compound'] <= -0.05 else "Neutral"))
Amazon.head()

In [ ]:
Google = tweet_info[(tweet_info.ticker_symbol == 'GOOGL') or (tweet_info.ticker_symbol == 'GOOG')]
Google['sentiment_scores'] = Google['body'].apply(lambda body: SentimentIntensityAnalyzer().polarity_scores(body))
Google['score'] = Google['sentiment_scores'].apply(lambda score: "Positive" if score['compound'] >= 0.05 else ("Negative" if score['compound'] <= -0.05 else "Neutral"))
Google.head()

In [ ]:
Tesla = tweet_info[(tweet_info.ticker_symbol == 'TSLA')]
Tesla['sentiment_scores'] = Tesla['body'].apply(lambda body: SentimentIntensityAnalyzer().polarity_scores(body))
Tesla['score'] = Tesla['sentiment_scores'].apply(lambda score: "Positive" if score['compound'] >= 0.05 else ("Negative" if score['compound'] <= -0.05 else "Neutral"))
Tesla.head()

In [ ]:
Microsoft = tweet_info[(tweet_info.ticker_symbol == 'MSFT')]
Microsoft['sentiment_scores'] = Microsoft['body'].apply(lambda body: SentimentIntensityAnalyzer().polarity_scores(body))
Microsoft['score'] = Microsoft['sentiment_scores'].apply(lambda score: "Positive" if score['compound'] >= 0.05 else ("Negative" if score['compound'] <= -0.05 else "Neutral"))
Microsoft.head()

In [ ]:
Apple.head()

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,total_engangement,sentiment_scores
0,550803612197457920,AAPL,SentiQuant,2015-01-01,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,0,1,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,550803610825928706,AAPL,SentiQuant,2015-01-01,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,0,1,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
5,550803225113157632,AAPL,MacHashNews,2015-01-01,Rumor Roundup: What to expect when you're expe...,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
6,550802957370159104,AAPL,WaltLightShed,2015-01-01,"An $AAPL store line in Sapporo Japan for the ""...",2,4,4,20,"{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp..."
7,550802855129382912,AAPL,2waystrading,2015-01-01,$AAPL - Will $AAPL Give Second entry opportuni...,0,0,0,0,"{'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'comp..."


In [ ]:
#Will need to calculate total score based on total_engagement and compound sentiment score

In [ ]:
#Convert to csv file for compilation. Comment out others if not compiled.
#After converting them into csv files, we saved them in the folder named "V.B.II.Company Visualizations_Dataframe" and use them in all codes of part V_B_ii

Apple.to_csv('/Users/../Apple.csv')
Amazon.to_csv('/Users/../Amazon.csv')
Google.to_csv('/Users/../Google.csv')
Tesla.to_csv('/Users/../Tesla.csv')
Microsoft.to_csv('/Users/../Microsoft.csv')